Goals: 

    1) Extract historical stock data from Yahoo (APPL, AMZN, MA, WMT, TSLA)
    
    2) Extract real-time stock data from Yahoo to make predictions (APPL, AMZN, MA, WMT, TSLA)

    3) Extract real-time stock data from Twitter (APPL, AMZN, MA, WMT, TSLA)
        - Run Sentiment Analysis on the Tweets and append the results to the input data
        
    4) Run ML algorithms on the data to make predictions about closing prices
        
    
    
    
        

### Imports

In [2]:
# Twitter and Sentiment Analysis
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud

# Data Processing and Requests
import websocket 
import json
import math
import pandas_datareader as web
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import requests_html
import lxml.html as lh
from datetime import datetime
from datetime import timedelta
import time
import unidecode #used to convert accented words
from lxml import html
import json
import argparse
from collections import OrderedDict

# SkLearn for pre-processing and Keras for LTSM Neural Network 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

Twitter data 

In [142]:
consumer_key = "0kmSS8yYrfOZDSEb8s6YpYatO"
consumer_secret = "R4Vpc52Bx1VQyyWipTV5peYGMbLBBzE9PjN4reN61Rpm55RGYm"
access_token = "1300816039803080705-Scvleu6b5nmcGfzQaL5rpTecalw7az"
access_token_secret = "A46HqzXW2SONU1xwqifrAsKI2o0HGv6WsTKlRRSttqX1j"

In [143]:
# Create the authentication object
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret) 
    
# Set the access token and access token secret
authenticate.set_access_token(access_token, access_token_secret) 
    
# Creating the API object while passing in auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)


In [144]:
# Extract 100 tweets from the twitter user
posts = api.user_timeline(screen_name="Visa", count = 100, lang ="en", tweet_mode="extended")


In [145]:
# Create a dataframe with a column called Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])
# Show the first 5 rows of data
df.head()

,Tweets
0,RT @VisaNews: Tap to pay is catching on as mor...
1,Committed to supporting Black women-owned smal...
2,Local businesses are the heart and soul of our...
3,RT @VisaNews: Consumers are going #digital and...
4,Read about the actions we’re taking to help st...


In [146]:
# Create a function to clean the tweets
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text


# Clean the tweets
df['Tweets'] = df['Tweets'].apply(cleanTxt)

# Show the cleaned tweets
df.head(5)

,Tweets
0,: Tap to pay is catching on as more consumers ...
1,Committed to supporting Black women-owned smal...
2,Local businesses are the heart and soul of our...
3,: Consumers are going digital and small busine...
4,Read about the actions we’re taking to help st...


In [147]:
# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


# Create two new columns 'Subjectivity' & 'Polarity'
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
df.head(5)

,Tweets,Subjectivity,Polarity
0,: Tap to pay is catching on as more consumers ...,0.577778,0.450000
1,Committed to supporting Black women-owned smal...,0.395833,0.083333
2,Local businesses are the heart and soul of our...,0.000000,0.000000
3,: Consumers are going digital and small busine...,0.180000,-0.100000
4,Read about the actions we’re taking to help st...,0.000000,0.000000


In [148]:
# Create a function to compute negative (-1), neutral (0) and positive (+1) analysis
def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'
df['Analysis'] = df['Polarity'].apply(getAnalysis)
# Show the dataframe
df.head(5)

,Tweets,Subjectivity,Polarity,Analysis
0,: Tap to pay is catching on as more consumers ...,0.577778,0.450000,Positive
1,Committed to supporting Black women-owned smal...,0.395833,0.083333,Positive
2,Local businesses are the heart and soul of our...,0.000000,0.000000,Neutral
3,: Consumers are going digital and small busine...,0.180000,-0.100000,Negative
4,Read about the actions we’re taking to help st...,0.000000,0.000000,Neutral
